In [18]:
# chamando o arquivo
import pandas as pd

df = pd.read_excel("processadores.xlsx")


In [19]:
# transformar string em float

# transformar os valores de strings em numeros

df['preço'] = df['preço'].str.replace('R$', '') 
df['preço'] = df['preço'].str.replace('.', '',)
df['preço'] = df['preço'].str.replace(',', '.') 
df['preço'] = df['preço'].str.strip() 
df['preço'] = df['preço'].astype(float)

print(df['preço'].head(10))

                        




0     749.99
1    1099.99
2    1749.99
3     649.99
4     749.99
5    1839.99
6     528.97
7    3249.99
8    2899.99
9    2539.99
Name: preço, dtype: float64


In [20]:
# biblioteca 

# dicionario com palavras
dicionario_palavras = {
    'Processadores': ['processador', 'processadores'],
    'Intel': ['intel', 'Intel']
}


In [21]:
# verifica se a palavra esta no dicionario

def contem_palavra_dicionario(texto, dicionario):
    texto = str(texto).lower()
    for palavras in dicionario.values():
        for palavra in palavras:
            if palavra.lower() in texto:
                return True
    return False

In [22]:
# armazena as palavras 

indices_para_remover = []

for i in df.index:
    texto = df.at[i, 'marca']
    if not contem_palavra_dicionario(texto, dicionario_palavras):
        indices_para_remover.append(i)

In [23]:
# apaga da tabela

df.drop(index=indices_para_remover, inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['marca'].head(200))

0      Processador Intel Core i5-12400F, 2.5GHz (4.4G...
1      Processador Intel Core i5-12400, 2.5GHz (4.4GH...
2      Processador Intel Core i5-14600K, 14ª Geração,...
3      Processador Intel Core i3-10105, 3.7GHz (4.4GH...
4      Processador Intel Core i5-10400F, 2.9GHz (4.3G...
                             ...                        
195    Processador Intel I7-3770, 3.90ghz, 8mb Cache ...
196    Processador Intel Core I5 - 10400F, 2.9GHz, Ca...
197    Processador Intel Core I5 14600k 14 Núcleos 20...
198    Processador Intel Core I3-7320 Pull Oem Socket...
199    Processador Intel Core I9-13900kf, 13ª Geração...
Name: marca, Length: 200, dtype: object


In [24]:
df.to_excel("processadores_limpo.xlsx", index=False)